In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_004 import *

In [ ]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'cifar10'

In [ ]:
data_mean,data_std = map(tensor, ([0.491, 0.482, 0.447], [0.247, 0.243, 0.261]))
cifar_norm = normalize_tfm(mean=data_mean,std=data_std)

In [ ]:
train_tfms = [flip_lr_tfm(p=0.5),
              pad_tfm(padding=4),
              crop_tfm(size=32, row_pct=(0,1.), col_pct=(0,1.)),
              cifar_norm]
valid_tfms = [cifar_norm]

In [ ]:
bs = 64

In [ ]:
#export
def to_half(b):  return [b[0].half(), b[1]]

In [ ]:
DataBunch

In [ ]:
#export
@dataclass
class DeviceDataLoader():
    dl: DataLoader
    device: torch.device
    progress_func: Callable
    half: bool = False
        
    def __len__(self): return len(self.dl)
    def __iter__(self):
        self.gen = (to_device(self.device,o) for o in self.dl)
        if self.half: self.gen = (to_half(o) for o in self.gen)
        if self.progress_func is not None:
            self.gen = self.progress_func(self.gen, total=len(self.dl), leave=False)
        return iter(self.gen)

    @classmethod
    def create(cls, *args, device=default_device, progress_func=tqdm, **kwargs):
        return cls(DataLoader(*args, **kwargs), device=device, progress_func=progress_func, half=False)

nb_002b.DeviceDataLoader = DeviceDataLoader

In [ ]:
train_ds = FilesDataset.from_folder(PATH/'train', classes=['airplane','dog'])
valid_ds = FilesDataset.from_folder(PATH/'test', classes=['airplane','dog'])
data = DataBunch.create(train_ds, valid_ds, bs=bs, train_tfm=train_tfms, valid_tfm=valid_tfms, num_workers=4)
len(data.train_dl), len(data.valid_dl)

In [ ]:
model = Darknet([1, 2, 2, 2, 2], num_classes=2, nf=16)
learn = Learner(data, model)
learn.metrics = [accuracy]
sched = OneCycleScheduler(learn, 0.1, 5)

# FP16

In [ ]:
#export
def bn2float(module):
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm): module.float()
    for child in module.children(): bn2float(child)
    return module

In [ ]:
#export
def model2half(model):
    """
    Converts the model to half precision except the batchnorm layers.
    """
    model = model.half()
    return bn2float(model)

Helper function to save the master model in FP32 with flat tensors (apparently it helps with performance)

In [ ]:
#export
from torch._utils import _flatten_dense_tensors, _unflatten_dense_tensors

In [ ]:
#export
def get_master(model, flat_master=False):
    """
    Returns two lists, one for the model parameters in FP16 and one for the master parameters in FP32
    """
    model_params = [param for param in model.parameters() if param.requires_grad]
    if flat_master:
        #Flattens all the parameters in one big tensor.
        master_params = _flatten_dense_tensors([param.data.float() for param in model_params])
        master_params = torch.nn.Parameter(master_params)
        master_params.requires_grad = True
        if master_params.grad is None: master_params.grad = master_params.new(*master_params.size())
        return model_params, [master_params]
    else:
        master_params = [param.clone().float().detach() for param in model_params]
        for param in master_params: param.requires_grad = True
        return model_params, master_params

In [ ]:
#export
def model_g2master_g(model_params, master_params, flat_master=False):
    """
    Copies the model gradients to the master parameters for the optimizer step.
    """
    if flat_master:
        master_params[0].grad.data.copy_(_flatten_dense_tensors([p.grad.data.float() for p in model_params]))
    else:
        for model, master in zip(model_params, master_params):
            if model.grad is not None:
                if master.grad is None: master.grad = master.data.new(*master.data.size())
                master.grad.data.copy_(model.grad.data)
            else: master.grad = None

In [ ]:
#export
def master2model(model_params, master_params, flat_master=False):
    """
    Copy master parameters to model parameters.
    """
    if flat_master:
        for model, master in zip(model_params, _unflatten_dense_tensors(master_params[0].data, model_params)):
            model.data.copy_(master)
    else:
        for model, master in zip(model_params, master_params):
            model.data.copy_(master.data)

In [ ]:
#export
class MixedPrecision(Callback):
    
    def __init__(self, learn, loss_scale=512, flat_master=False):
        self.learn,self.loss_scale,self.flat_master = learn,loss_scale, flat_master
    
    def on_train_begin(self, **kwargs):
        #Insures the dataloaders are in half precision.
        self.learn.data.train_dl.half = True
        if hasattr(self.learn.data, 'valid_dl') and self.learn.data.valid_dl is not None:
            self.learn.data.valid_dl.half = True
        #Get a copy of the model params in FP32
        self.model_params, self.master_params = get_master(self.learn.model, self.flat_master)
        #Changes the optimizer so that the optimization step is done in FP32.
        opt = self.learn.opt
        mom,wd,beta = opt.mom,opt.wd,opt.beta
        self.learn.opt.opt = self.learn.opt_fn(self.master_params, self.learn.opt.lr)
        opt.mom,opt.wd,opt.beta = mom,wd,beta
    
    def on_loss_begin(self, last_output, **kwargs):
        #It's better to compute the loss in FP32, to avoid reduction overflow.
        return last_output.float()
    
    def on_backward_begin(self, last_loss, **kwargs):
        #To avoid gradient underflow, we scale the gradients
        return last_loss * self.loss_scale
    
    def on_backward_end(self, **kwargs):
        #Convert the gradients back to FP32 and divide them by the scale.
        model_g2master_g(self.model_params, self.master_params, self.flat_master)
        for param in self.master_params: param.grad.div_(self.loss_scale)
        #Zeros the gradients of the model since the optimizer is disconnected.
        self.learn.model.zero_grad()
    
    def on_step_end(self, **kwargs):
        #Update the params from master to model.
        master2model(self.model_params, self.master_params, self.flat_master)

In [ ]:
model = Darknet([1, 2, 2, 2, 2], num_classes=2, nf=16)
model = model2half(model)
learn = Learner(data, model)
learn.metrics = [accuracy]
scheds = [MixedPrecision(learn, flat_master=True), OneCycleScheduler(learn, 0.1, 5)]

In [ ]:
learn.fit(1, 1e-2, callbacks=scheds)